In [2]:
import sys, pickle, os
import math, json, time
import decimal
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from random import shuffle
from copy import deepcopy
from sklearn import preprocessing

In [3]:
import keras
from keras.models import Model
from keras.preprocessing import sequence
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Add
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D, GRU
from keras.layers import Input, Embedding, LSTM, Dense, TimeDistributed, Masking, RepeatVector, merge, Flatten
from keras.models import Model
from keras.utils import plot_model
from keras.layers import Bidirectional
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import optimizers, layers

Using TensorFlow backend.


In [42]:
XDinput = Input(shape=(1000,), dtype='int32') ### Buralar flagdan gelmeliii
XTinput = Input(shape=(400,), dtype='int32')
NUM_FILTERS = 32
FILTER_LENGTH1 = 16
FILTER_LENGTH2 = 16

In [43]:
encode_smiles = Embedding(input_dim=1000+1, output_dim=128, input_length=1000)(XDinput)
encode_smiles = Conv1D(filters=NUM_FILTERS, kernel_size=FILTER_LENGTH1,  activation='relu', padding='valid',  strides=1)(encode_smiles)
encode_smiles = Conv1D(filters=NUM_FILTERS*2, kernel_size=FILTER_LENGTH1,  activation='relu', padding='valid',  strides=1)(encode_smiles)
encode_smiles = Conv1D(filters=NUM_FILTERS*3, kernel_size=FILTER_LENGTH1,  activation='relu', padding='valid',  strides=1)(encode_smiles)
encode_smiles = GlobalMaxPooling1D()(encode_smiles)


encode_protein = Embedding(input_dim=400+1, output_dim=128, input_length=400)(XTinput)
encode_protein = Conv1D(filters=NUM_FILTERS, kernel_size=FILTER_LENGTH2,  activation='relu', padding='valid',  strides=1)(encode_protein)
encode_protein = Conv1D(filters=NUM_FILTERS*2, kernel_size=FILTER_LENGTH2,  activation='relu', padding='valid',  strides=1)(encode_protein)
encode_protein = Conv1D(filters=NUM_FILTERS*3, kernel_size=FILTER_LENGTH2,  activation='relu', padding='valid',  strides=1)(encode_protein)
encode_protein = GlobalMaxPooling1D()(encode_protein)

XEInput = Input(shape=(500,), dtype='float32')
# encode_energy = GlobalMaxPooling1D()(XEInput)

encode_interaction = keras.layers.concatenate([encode_smiles, encode_protein], axis=-1) #merge.Add()([encode_smiles, encode_protein])

# Fully connected
FC1 = Dense(1024, activation='relu')(encode_interaction)
# FC1 = keras.layers.concatenate([FC1,XEInput], axis=-1)
FC2 = Dropout(0.1)(FC1)
FC2 = Dense(1024, activation='relu')(FC2)
FC2 = Dropout(0.1)(FC2)
FC2 = Dense(512, activation='relu')(FC2)

predictions = Dense(1, kernel_initializer='normal')(FC2)

In [32]:
# predictions = Dense(1, kernel_initializer='normal')(FC2)

In [44]:
interactionModel = Model(inputs=[XDinput, XTinput], outputs=[predictions])

interactionModel.compile(optimizer='adam', loss='mean_squared_error') #, metrics=['cindex_score']
print(interactionModel.summary())
plot_model(interactionModel, to_file='figures/build_combined_categorical.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_23 (InputLayer)           (None, 1000)         0                                            
__________________________________________________________________________________________________
input_24 (InputLayer)           (None, 400)          0                                            
__________________________________________________________________________________________________
embedding_30 (Embedding)        (None, 1000, 128)    128128      input_23[0][0]                   
__________________________________________________________________________________________________
embedding_31 (Embedding)        (None, 400, 128)     51328       input_24[0][0]                   
__________________________________________________________________________________________________
conv1d_94 

In [47]:
XEInput = Input(shape=(500,1,), dtype='float32')

energy_mat = Conv1D(filters=NUM_FILTERS, kernel_size=FILTER_LENGTH1,  activation='relu', padding='valid')(XEInput)
encode_energy = GlobalMaxPooling1D()(energy_mat)

energy_mat = Dense(10, activation='relu')(encode_energy)


FC3 = keras.layers.concatenate([FC2, energy_mat], axis=-1)
pred_prev = Dense(100, kernel_initializer='normal')(FC3)
pred_prev =  Dropout(0.1)(pred_prev)
pred = Dense(1, kernel_initializer='normal')(pred_prev)

newModel = Model(inputs=[XDinput, XTinput, XEInput], outputs=[pred] )
newModel.compile(optimizer='adam', loss='mean_squared_error')
print(newModel.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_23 (InputLayer)           (None, 1000)         0                                            
__________________________________________________________________________________________________
input_24 (InputLayer)           (None, 400)          0                                            
__________________________________________________________________________________________________
embedding_30 (Embedding)        (None, 1000, 128)    128128      input_23[0][0]                   
__________________________________________________________________________________________________
embedding_31 (Embedding)        (None, 400, 128)     51328       input_24[0][0]                   
__________________________________________________________________________________________________
conv1d_94 